# National Surface Depressions

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/source-coop-readme/blob/main/depressions/Depressions.ipynb)

## Description

This dataset represents the extent and location of surface depressions derived from the 10-m resolution dataset from the 3D Elevation Program ([3DEP](https://www.usgs.gov/3d-elevation-program)). The levet-set algorithm available through the [lidar](https://lidar.gishub.org) Python package was used the process the DEM and delineate surface depressions at the HU8 watershed scale, which was then merged to create a depression dataset by state. The dataset is available [here](https://beta.source.coop/repositories/giswqs/depressions) in the form of GeoParquet files.

## Reference

- Wu, Q., Lane, C. R., Wang, L., Vanderhoof, M. K., Christensen, J. R., & Liu, H. (2019). Efficient Delineation of Nested Depression Hierarchy in Digital Elevation Models for Hydrological Analysis Using Level‐Set Method. JAWRA Journal of the American Water Resources Association , 55(2), 354-368. https://doi.org/10.1111/1752-1688.12689

## Installation

In [ ]:
%pip install duckdb leafmap

## Import libraries

In [ ]:
import duckdb
import leafmap
import matplotlib.pyplot as plt

In [ ]:
con = duckdb.connect()
con.install_extension("spatial")
con.load_extension("spatial")

## Analyze individual states

For example, we can analyze the surface depressions in the state of Tennessee.

In [ ]:
state = "TN"  # Change to the state of your choice
url = f"https://data.source.coop/giswqs/depressions/state/{state}.parquet"
con.sql(f"SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) as geometry FROM '{url}'")

Return the results as a Pandas DataFrame.

In [ ]:
state = "TN"  # Change to the state of your choice
url = f"https://data.source.coop/giswqs/depressions/state/{state}.parquet"
df = con.sql(f"SELECT * EXCLUDE geometry FROM '{url}'").df()
df.head()

Calculate summary statistics for the surface depressions in the selected state.

In [ ]:
df.describe()

Create a histogram of the surface depression areas in the selected state.

In [ ]:
plt.hist(df["area"], bins=15, edgecolor="black", range=(1000, 20000))
plt.xlabel("Area (sq. meters)")
plt.ylabel("Frequency")
plt.title("Histogram of Depression Area")
plt.show()

Create a scatter plot of the surface depression volumes in the selected state.

In [ ]:
plt.hist(df["volume"], bins=15, edgecolor="black", range=(1000, 20000))
plt.xlabel("Volume (cubic meters)")
plt.ylabel("Frequency")
plt.title("Histogram of Depression Volume")
plt.show()

Create a scatter plot of the surface depression mean depth in the selected state.

In [ ]:
plt.hist(df["avg-depth"], bins=15, edgecolor="black", range=(0, 1))
plt.xlabel("Depth (meter)")
plt.ylabel("Frequency")
plt.title("Histogram of Depression Mean Depth")
plt.show()

## Analyze all states

Calculate the total number of surface depressions in the United States, including lower 48 states, Alaska, and Hawaii.

In [ ]:
url = "s3://us-west-2.opendata.source.coop/giswqs/depressions/state/*.parquet"
con.sql(
    f"""
SELECT COUNT(*) AS Count
FROM '{url}'
"""
)

Find out the number of surface depressions in each state. Note that the datasets do not contain a field for state names. The `filename` argument can be used to add an extra filename column to the result that indicates which row came from which file.

In [ ]:
df = con.sql(
    f"""
SELECT filename, COUNT(*) AS Count
FROM read_parquet('s3://us-west-2.opendata.source.coop/giswqs/depressions/state/*.parquet', filename=true)
GROUP BY filename
ORDER BY COUNT(*) DESC;
"""
).df()
df.head()

Inspect the list of filenames.

In [ ]:
df["filename"].tolist()

Create a `State` column based on the `filename` column by extracting the state name from the filename.

In [ ]:
count_df = con.sql(
    f"""
SELECT SUBSTRING(filename, LENGTH(filename) - 9, 2) AS State, COUNT(*) AS Count
FROM read_parquet('s3://us-west-2.opendata.source.coop/giswqs/depressions/state/*.parquet', filename=true)
GROUP BY State
ORDER BY COUNT(*) DESC;
"""
).df()
count_df.head(10)

Create a bar plot of the number of surface depressions in each state.

In [ ]:
leafmap.bar_chart(count_df, x="State", y="Count", title="Depression Count by State")

Create a pie chart of the number of surface depressions in each state.

In [ ]:
leafmap.pie_chart(count_df, names="State", values="Count", height=600)

Create a depressions table from the DataFrame above.

In [ ]:
con.sql("CREATE OR REPLACE TABLE depressions AS FROM count_df")
con.sql("FROM depressions")

To visualize the data on the map, we need a GeoDataFrame. Let's create a states table from the [us_states.parquet](https://open.gishub.org/data/us/us_states.parquet) file.

In [ ]:
url = "https://open.gishub.org/data/us/us_states.parquet"
con.sql(
    f"""
CREATE OR REPLACE TABLE states AS
SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) 
AS geometry FROM '{url}'
"""
)
con.sql("FROM states")

Join the `depressions` table with the `states` table.

In [ ]:
con.sql(
    """
SELECT * FROM states INNER JOIN depressions ON states.id = depressions.State
"""
)

Export the joined table as a Pandas DataFrame.

In [ ]:
df = con.sql(
    """
SELECT name, State, Count, ST_AsText(geometry) as geometry
FROM states INNER JOIN depressions ON states.id = depressions.State
"""
).df()
df.head()

Convert the Pandas DataFrame to a GeoDataFrame.

In [ ]:
gdf = leafmap.df_to_gdf(df, src_crs="EPSG:4326")

Visualize the data on the map.

In [ ]:
m = leafmap.Map(center=[40, -100], zoom=4)
m.add_data(
    gdf,
    column="Count",
    scheme="Quantiles",
    cmap="Greens",
    legend_title="Depression Count",
)
m

Calculate summary statistics for the surface depression areas in the United States.

In [ ]:
con.sql(
    f"""
SELECT SUM(area) /  1000000 AS Sum_Area_SqKm, MIN(area) AS Min_Area, MAX(area) AS Max_Area, AVG(area) AS Avg_Area, STDDEV(area) AS Std_Area, MEDIAN(area) AS Median_Area
FROM 's3://us-west-2.opendata.source.coop/giswqs/depressions/state/*.parquet'
"""
)

Calculate summary statistics for the surface depression volumes in the United States.

In [ ]:
con.sql(
    f"""
SELECT SUM(volume) /  1000000000 AS Sum_Volume_CuKm, MAX(volume) AS Max_Volume, AVG(volume) AS Avg_Volume, STDDEV(volume) AS Std_Volume, MEDIAN(volume) AS Median_Volume
FROM 's3://us-west-2.opendata.source.coop/giswqs/depressions/state/*.parquet'
"""
)

Calculate summary statistics for the surface depression mean depths in the United States.

In [ ]:
con.sql(
    f"""
SELECT AVG("avg-depth") AS Avg_Depth, STDDEV("avg-depth") AS Std_Depth, MEDIAN("avg-depth") AS Median_Depth
FROM 's3://us-west-2.opendata.source.coop/giswqs/depressions/state/*.parquet'
"""
)

Calculate the total area of surface depressions by state.

In [ ]:
area_df = con.sql(
    f"""
SELECT SUBSTRING(filename, LENGTH(filename) - 9, 2) AS State, SUM(area) /  1000000 AS Area_SqKm, MEDIAN(area) AS Median_Area
FROM read_parquet('s3://us-west-2.opendata.source.coop/giswqs/depressions/state/*.parquet', filename=true)
GROUP BY State
ORDER BY COUNT(*) DESC;
"""
).df()
area_df.head(10)

Create a bar plot of the total area of surface depressions by state.

In [ ]:
leafmap.bar_chart(
    area_df, x="State", y="Area_SqKm", title="Total Depression Area by State (Sq. Km)"
)

In [ ]:
leafmap.bar_chart(
    area_df, x="State", y="Median_Area", title="Median Depression Area by State (m2)"
)

Create a pie chart of the total area of surface depressions by state.

In [ ]:
leafmap.pie_chart(area_df, names="State", values="Area_SqKm", height=750)

Calculate the total volume of surface depressions by state.

In [ ]:
volume_df = con.sql(
    f"""
SELECT SUBSTRING(filename, LENGTH(filename) - 9, 2) AS State, SUM(volume) /  1000000000 AS Volume_CuKm, MEDIAN(volume) AS Median_Volume
FROM read_parquet('s3://us-west-2.opendata.source.coop/giswqs/depressions/state/*.parquet', filename=true)
GROUP BY State
ORDER BY COUNT(*) DESC;
"""
).df()
volume_df.head(10)

Create a bar plot of the total volume of surface depressions by state.

In [ ]:
leafmap.bar_chart(
    volume_df,
    x="State",
    y="Volume_CuKm",
    title="Total Depression Volume by State (Cu. Km)",
)

In [ ]:
leafmap.bar_chart(
    volume_df, x="State", y="Median_Volume", title="Median Depression Volume by State"
)

Create a pie chart of the total volume of surface depressions by state.

In [ ]:
leafmap.pie_chart(volume_df, names="State", values="Volume_CuKm", height=900)